In [ ]:
%pip install plotly
%pip install pandas
%pip install nbformat --upgrade
%pip install numpy
%pip install lifetimes
%pip install --upgrade matplotlib
%pip install seaborn
# Install squarify for treemap plot
%pip install squarify
'''pip install numpy
pip install lifetimes
pip install --upgrade matplotlib
pip install seaborn
pip install scikit-learn'''

In [ ]:

import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import squarify


# Data Wrangling

In [ ]:
data = pd.read_csv('C:\\Users\\Admin\\OneDrive\\Desktop\\Jupyter\\Online Retail\\online_retail.csv')
data.head()

In [ ]:
data.tail()

In [ ]:
data.dropna(subset = ['CustomerID'], inplace = True)

In [ ]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['TotalAmount'] = (data['Quantity'] * data['UnitPrice']).astype('Float64').round(2)

In [ ]:
data.head()

In [ ]:
reference_date = pd.Timestamp(dt.now().date())
#reference_date

In [ ]:
reference_date = data['InvoiceDate'].max() + timedelta(days = 1)
reference_date

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
data.shape

In [ ]:
#number of unique products
print(data['Description'].nunique())

In [ ]:
#number of each products
data['Description'].value_counts().head()

In [ ]:
#The most ordered product
data.groupby("Description").agg({"Quantity" : "sum"}).head()


In [ ]:
#Number of invoices were issued in total
data['InvoiceNo'].nunique()

In [ ]:
# Total Price that was paid per invoice
## return Price to float
data['TotalAmount']

In [ ]:
data.describe()

In [ ]:
# คอลัมน์ Price , Quantity มีข้อมูลที่เป็นลบ
print(f"Total 'Quantity' column less than 0: {np.sum(data['Quantity'] <= 0)} row")
print(f"Total 'TotalAmount' column less than 0: {np.sum(data['TotalAmount'] <= 0)} row")

In [ ]:
#data[data['Quantity'] <= 0]
data[data['TotalAmount'] <= 0]

In [ ]:
data = data[data['Quantity'] >= 0]

In [ ]:
data.describe()

In [ ]:
# The maximum and minimum of 'Quantity' and 'UnitPrice' are too far
# Replace with quartile method
for col in ['Quantity', 'UnitPrice']:
    data_quartile = sorted(data[col])
    Q1, Q3 = np.percentile(data_quartile, [0.01, 0.99])
    IQR = Q3 - Q1
    lower_limit = Q1 - (1.5 * IQR)
    upper_limit = Q3 + (1.5 * IQR)
    data[data_quartile] = np.where(data[col] > upper_limit, upper_limit, data[col])
    data[data_quartile] = np.where(data[col] < lower_limit, lower_limit, data[col])


In [ ]:
data.describe()

# RFM Model 
'''
Recency : ลูกค้าที่สั่งซื้อสินค้าล่าสุดวันนี้ จะได้ R Score > ลูกค้าที่สั่งซื้อสินค้าล่าสุดเมื่อ 3 ปีที่แล้ว
Frequency : ลูกค้าที่สั่งซื้อสินค้าบ่อยที่สุดในร้าน จะได้ F Score > ลูกค้าmujสั่งซื้อสินค้าน้อยครั้งที่สุดในร้าน
Monetary : ลูกค้าที่ช้จ่ายเยอะที่สุดในร้าน จะได้ M Score > ลูกค้าที่ใช้จ่ายน้อยที่สุดในร้าน
'''

In [ ]:
#ทำRFM Analysis โดย group by Customer ID
rfm = data.groupby('CustomerID').agg({
    'InvoiceDate' : lambda x: (reference_date - x.max()).days,
    'InvoiceNo' : 'count',
    'TotalAmount' : 'sum'
})

In [ ]:
#ทำตารางRFM
rfm.rename(columns = {'InvoiceDate': 'Recency', 'InvoiceNo' : 'Frequency',
                      'TotalAmount' : 'Monetary'}, inplace = True)
rfm.head()

In [ ]:
# Create binning for 5 bins
rfm['R'] = pd.qcut(rfm['Recency'], 5, labels = [5, 4, 3, 2, 1])
rfm['F'] = pd.qcut(rfm['Frequency'].rank(method = 'first'),
                                  5, labels = [1, 2, 3, 4, 5])
rfm['M'] = pd.qcut(rfm['Monetary'], 5, labels = [5, 4, 3, 2, 1])
rfm.head(5)

In [ ]:
#ระบุ Quantiles
'''
quantiles = rfm.quantile(q = [0.25, 0.50, 0.75])

#จัดคะแนนตามRFM
# x คือ ค่าที่จะกลายเป็นscore ในRFM
# p คือ column name ในRFM
# d คือ data frame ที่เก็บquantiles
def RScore(x, p, d):
    #Recency
    if (p == 'Recency'):
        if (x <= d[p][0.25]):
            return 4
        elif (x <= d[p][0.50]):
            return 3
        elif (x <= d[p][0.75]):
            return 2
        else:
            return 1
    # Frequency, Monetery
    else:
        if (x <= d[p][0.25]):
            return 1
        elif (x <= d[p][0.50]):
            return 2
        elif (x <= d[p][0.75]):
            return 3
        else:
            return 4


rfm['R'] = rfm['Recency'].apply(RScore, args = ('Recency', quantiles, ))
rfm['F'] = rfm['Frequency'].apply(RScore, args = ('Frequency', quantiles, ))
rfm['M'] = rfm['Monetary'].apply(RScore, args = ('Monetary', quantiles, ))
'''

In [ ]:
rfm.head()

In [ ]:
rfm['RFM_Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].sum(axis = 1)

In [ ]:

# RFM Score
rfm['rfmSegment'] = rfm['R'].astype(str) + rfm['F'].astype(str)
#rfm.drop(columns=['RFMScore'], inplace=True)
rfm.head(5)

In [ ]:
# Convert RFM Score to segment label
rfmLabel = {
    r'55': "Champion",
    r'[3-5][4-5]': "Loyal",
    r'[4-5][2-3]': "Promising",
    r'51': "New Customers",
    r'41': "Warm Leads",
    r'31': "Cold Leads",
    r'[2-3][2-3]': "Need Attention",
    r'[1-2][5]': "Shouldn't Lost",
    r'[1-2][3-4]': 'Sleepers',
    r'[1-2][1-2]': "Lost"
}

rfm['Segment'] = rfm['rfmSegment'].replace(rfmLabel, regex = True)

In [ ]:
rfm.head()

In [ ]:
# RFM Result
result = rfm.groupby(['Segment'])['Segment'].count()
values = list(result)
labels = result.index

# Plot
colors = [list(np.random.uniform(size = 4)) for i in range(len(values))]

plt.figure(figsize = (18, 11))
with plt.style.context('ggplot'):
  squarify.plot(sizes = values, color = colors, label = labels)
  plt.axis('off')
  plt.title('Customer segmentation')
  plt.show()

In [ ]:
rfm.head()

In [ ]:
vip_segment = rfm[rfm['Segment'] == "Loyal"]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y = vip_segment['Recency'], name = "Recency"))
fig.add_trace(go.Box(y = vip_segment['Frequency'], name = "Frequency"))
fig.add_trace(go.Box(y = vip_segment['Monetary'], name = "Monetary"))


In [ ]:
correlation_matrix = vip_segment[['R', 'F', 'M']].corr()

In [ ]:
#Create Heatmap

fig_heatmap = go.Figure(data = go.Heatmap(
                    z = correlation_matrix.values,
                    x = correlation_matrix.columns,
                    y = correlation_matrix.columns,
                    colorscale = 'RdBu',
                    colorbar = dict(title = 'Correlation')))

fig_heatmap.update_layout(title = "Correlation Matrix of RFM Monetary within Loyal Segment")


#Show Heatmap

fig_heatmap.show()

In [ ]:
data.to_csv("Cleaned_Data.csv", index = False)